In [2]:
using Revise

using RigidBodyDynamics
using RigidBodyDynamics: Bounds

using Plots
using BilevelTrajOpt

In [3]:
urdf = joinpath("..", "urdf", "ball.urdf")
mechanism = parse_urdf(Float64, urdf)
body = findbody(mechanism, "ball")
basejoint = joint_to_parent(body, mechanism)
floatingjoint = Joint(basejoint.name, frame_before(basejoint), frame_after(basejoint), QuaternionFloating{Float64}())
replace_joint!(mechanism, basejoint, floatingjoint)
position_bounds(floatingjoint) .= Bounds(-100, 100)
velocity_bounds(floatingjoint) .= Bounds(-100, 100)
μ = 0.5
motion_type = :xyz
point = Point3D(default_frame(findbody(mechanism, "floor")), [0.,0.,0.])
normal = FreeVector3D(default_frame(findbody(mechanism, "floor")), [0.,0.,1.])
hs = HalfSpace(point, normal)
floor = Obstacle(hs, μ, motion_type)
obstacles = [floor]
env = parse_contacts(mechanism, urdf, obstacles)
x0 = MechanismState(mechanism)
Δt = 0.005
sim_data = get_sim_data(x0,env,Δt,true);

In [34]:
q0 = [1., 0., 0., 0., 0., 0., 0.]
v0 = [0., 0., 0., 0.5, 0., 0.]

u0 = zeros(sim_data.num_v)
set_configuration!(x0, q0)
set_velocity!(x0, v0)
setdirty!(x0)
ctrl! = (u,t,x) -> u[:] = 0.
traj = BilevelTrajOpt.simulate(x0,env,sim_data.Δt,1,ctrl!,implicit_contact=false)
qnext = traj[1:sim_data.num_q,2]
vnext = traj[sim_data.num_q+1:sim_data.num_q+sim_data.num_v,2]
x_sol_exp = traj[sim_data.num_q+sim_data.num_v+2:end,2]

# qnext = q0
# vnext = v0

τ_ip, x_sol_ip, λ_ip, μ_ip, fx_ip = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=true);

τ_auglag, x_sol_auglag, λ_auglag, μ_auglag, fx_auglag = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=false);

# display("Explicit")
# display(x_sol_exp)
display("IP")
display(x_sol_ip)
display("Aug Lag")
display(x_sol_auglag)

display("torques")
display(τ_ip)
display(τ_auglag)

elapsed time: 0.004895411 seconds
elapsed time: 0.004723595 seconds
elapsed time: 0.004916079 seconds
elapsed time: 0.004799368 seconds
elapsed time: 0.004902464 seconds
elapsed time: 0.004783138 seconds
elapsed time: 0.004955154 seconds
elapsed time: 0.004799098 seconds
elapsed time: 0.004894248 seconds
elapsed time: 0.004831199 seconds
elapsed time: 0.004802805 seconds
elapsed time: 0.005190149 seconds
elapsed time: 0.004700602 seconds
elapsed time: 

0.042758683 seconds
elapsed time: 0.005076744 seconds
elapsed time: 0.004817373 seconds
elapsed time: 0.006095974 seconds
Solve_Succeeded


"IP"

6-element Array{Float64,1}:
 0.0       
 5.41078e-8
 0.500003  
 5.41078e-8
 0.483278  
 1.00001   

"Aug Lag"

6-element Array{Float64,1}:
 0.104737
 0.11044 
 0.116087
 0.13106 
 0.0     
 0.508543

"torques"

6-element Array{Float64,1}:
  0.0        
  0.0        
  0.0        
  4.90503    
 -6.00693e-16
 -9.81006    

6-element Array{Float64,1}:
  0.0     
  0.0     
  0.0     
  0.111346
  0.202278
 -4.9888  

Solve_Succeeded


In [31]:
τ_auglag, x_sol_auglag, λ_auglag, μ_auglag, fx_auglag = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=false);

# autodiff 
J_auto = ForwardDiff.jacobian(qv -> solve_implicit_contact_τ(sim_data,q0,v0,u0,qv[1:sim_data.num_q],qv[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=false)[2], vcat(qnext,vnext))
Jt_auto = ForwardDiff.jacobian(qv -> solve_implicit_contact_τ(sim_data,q0,v0,u0,qv[1:sim_data.num_q],qv[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=false)[1], vcat(qnext,vnext))

# numerical
ϵ = 1e-12
J_num = zeros(size(J_auto))
Jt_num = zeros(size(J_auto))
Jt_num_ip = zeros(size(J_auto))
qv = vcat(qnext,vnext)
for i = 1:length(qv)
    δ = zeros(length(qv))
    δ[i] = ϵ
    τ, x, λ, μ, fx = solve_implicit_contact_τ(sim_data,q0,v0,u0,(qv+δ)[1:sim_data.num_q],(qv+δ)[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=false)
    J_num[:,i] = (x - x_sol_auglag)/ϵ
    Jt_num[:,i] = (τ - τ_auglag)/ϵ
    τ, x, λ, μ, fx = solve_implicit_contact_τ(sim_data,q0,v0,u0,(qv+δ)[1:sim_data.num_q],(qv+δ)[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=true)
    Jt_num_ip[:,i] = (τ - τ_auglag)/ϵ
end

display(J_auto)
display(J_num)

display(Jt_auto)
display(Jt_num)
display(Jt_num_ip)

Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded


6×13 Array{Float64,2}:
 0.0   3.92198e-12   0.0523031   -2.94345e-6  …   1.31078e-6   1.06628   
 0.0  -4.5477e-6     0.0734589   -0.262876        5.35923      1.49763   
 0.0   4.15333e-12   0.0659371   -3.07586e-6      1.61538e-6   1.34438   
 0.0   4.54771e-6    0.0734589    0.262865       -5.35922      1.49763   
 0.0  -6.00741e-12  -1.51201e-7   4.78868e-6     -2.01635e-6   3.52857e-6
 0.0   5.15305e-13   0.731066    -1.99696e-6  …   1.20405e-6  14.9044    

6×13 Array{Float64,2}:
 0.0   3.78944e7      2.80567e7  …  -4.43347e5   4.37528e7   8.26287e7
 0.0  -1.30823e7      2.40254e6     -3.19569e6   2.76064e6  -2.79234e6
 0.0  -1.12841e8     -7.51169e7      8.84663e6  -1.13342e8  -1.67894e8
 0.0   1.93767e7      4.45482e6      4.53956e6   1.13747e7   3.27449e7
 0.0   1.08453e7      4.729e6       -1.60158e6   7.87118e6   1.43117e6
 0.0  -1.03366e7  96937.7        …   5.04222e6  -7.04132e6  -4.50398e6

6×13 Array{Float64,2}:
 0.0    0.0          -3.08149e-33  …     0.0            0.0        
 0.0    0.0           0.0                0.0            0.0        
 0.0    0.0           0.0                0.0            0.0        
 0.0    2.26955e-12  25.9001             2.98818e-6     2.72816    
 0.0  -25.7662       -1.36141e-16     -105.148         -4.35652e-15
 0.0   -5.04426e-12  -1.90877      …    -1.18117e-5  -146.212      

6×13 Array{Float64,2}:
 0.0   0.0         0.0        …   0.0         0.0         0.0      
 0.0   0.0         0.0            0.0         0.0         0.0      
 0.0   0.0         0.0            0.0         0.0         0.0      
 0.0  -1.47872e9  -1.01213e9      9.11347e7  -1.5411e9   -2.45762e9
 0.0   3.18423e8   2.01328e7      7.58828e7   8.45038e7   3.4862e8 
 0.0   1.01402e8  -9.50953e5  …  -4.94642e7   6.90754e7   4.4184e7 

6×13 Array{Float64,2}:
 0.0         0.0         0.0         …  0.0         0.0         0.0       
 0.0         0.0         0.0            0.0         0.0         0.0       
 0.0         0.0         0.0            0.0         0.0         0.0       
 2.27273e12  2.27273e12  2.27273e12     2.27273e12  2.27273e12  2.27273e12
 2.09097e8   2.09097e8   2.09097e8      2.09097e8   2.09096e8   2.09097e8 
 3.07309e12  3.07309e12  3.07309e12  …  3.07309e12  3.07309e12  3.07309e12

Solve_Succeeded


In [33]:
display(Jt_auto[:,11:13])
display(Jt_num[:,11:13])
display(Jt_num_ip[:,11:13])

6×3 Array{Float64,2}:
    0.0            0.0            0.0        
    0.0            0.0            0.0        
    0.0            0.0            0.0        
 -109.081          2.98818e-6     2.72816    
    0.0         -105.148         -4.35652e-15
   -0.00013461    -1.18117e-5  -146.212      

6×3 Array{Float64,2}:
  0.0         0.0         0.0      
  0.0         0.0         0.0      
  0.0         0.0         0.0      
  9.11347e7  -1.5411e9   -2.45762e9
  7.58828e7   8.45038e7   3.4862e8 
 -4.94642e7   6.90754e7   4.4184e7 

6×3 Array{Float64,2}:
 0.0         0.0         0.0       
 0.0         0.0         0.0       
 0.0         0.0         0.0       
 2.27273e12  2.27273e12  2.27273e12
 2.09097e8   2.09096e8   2.09097e8 
 3.07309e12  3.07309e12  3.07309e12

In [23]:
 (J_num[:,1] - x_sol_auglag)/ϵ

6-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [35]:
maximum(abs.(J_auto-J_num))

0.00011239110444094536

In [12]:
abs.(J_auto-J_num)

6×13 Array{Float64,2}:
 0.0  1.17929e8    5.54961e7    5.90191e7    0.0  …  4.85598e7   0.115786   
 0.0  1.17929e8    5.54961e7    5.90191e7    0.0     4.85598e7   0.115791   
 0.0  1.17929e8    5.54961e7    5.90191e7    0.0     4.85598e7   0.115786   
 0.0  1.17929e8    5.54961e7    5.90191e7    0.0     4.85598e7   0.115777   
 0.0  5.52014e-42  4.60942e-39  9.22638e-47  0.0     2.60672e-6  1.73472e-6 
 0.0  9.16002e-25  8.60679e-24  5.07938e-39  0.0  …  4.19766e-5  0.000109654

In [27]:
J_num

6×13 Array{Float64,2}:
 0.0  -5.53932e7   2.60583e7   1.46989e7  0.0  0.0  …   1.35127e7     2.76304
 0.0   5.54017e7  -2.60562e7  -1.46982e7  0.0  0.0     -1.35121e7     1.19871
 0.0  -5.53932e7   2.60583e7   1.46989e7  0.0  0.0      1.35127e7     2.76307
 0.0   5.54017e7  -2.60562e7  -1.46982e7  0.0  0.0     -1.35121e7     1.19871
 0.0   0.0         0.0         0.0        0.0  0.0      0.0           0.0    
 0.0   0.0         0.0         0.0        0.0  0.0  …  -0.000444089  20.3861 

In [28]:
J_auto

6×13 Array{Float64,2}:
 0.0  -3.30962e-15  -5.87669e-15   3.27191e-6   …   2.34824e-5   1.98036    
 0.0   3.3259e-15    5.91148e-15  -3.27238e-6       7.92411      1.98142    
 0.0  -3.30962e-15  -5.94641e-15   3.27191e-6       2.34824e-5   1.98036    
 0.0   3.29125e-15   5.91148e-15  -3.27238e-6      -7.92412      1.98142    
 0.0  -8.98363e-32  -1.21529e-31  -1.78426e-38      2.28571e-6   3.38714e-21
 0.0  -7.82746e-18  -4.70622e-18  -5.10436e-32  …   8.23815e-5  20.3864     